<a href="https://colab.research.google.com/github/ZahraDehghanian97/LenseCraft/blob/master/Cinematography_Instruction_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/

from getpass import getpass

repo_url = f"https://{getpass('GitHub Username: ')}:{getpass('GitHub Personal Access Token: ')}@github.com/ZahraDehghanian97/LensCraft.git"

!git clone --recurse-submodules -j8 {repo_url}

%cd /content/LensCraft

!pip install -r requirements.txt

!mkdir -p datasets/simulation
!gdown -O datasets/simulation/random_simulation_dataset.json 1VT2XfBj9LFWLUBjv65dzC4bVzH0zdNDU

In [ ]:
%cd /content/LensCraft
!python src/train.py batch_size=16